In [1]:
import sys
from pathlib import Path

# Utiliser cwd() + parent
project_root = Path.cwd().parent  # notebooks/.. = racine
sys.path.append(str(project_root / "src"))

# Vérifier
print(sys.path)

['C:\\Program Files\\Python313\\python313.zip', 'C:\\Program Files\\Python313\\DLLs', 'C:\\Program Files\\Python313\\Lib', 'C:\\Program Files\\Python313', 'c:\\Users\\blanc\\code\\Supply_chain\\env_project', '', 'c:\\Users\\blanc\\code\\Supply_chain\\env_project\\Lib\\site-packages', 'c:\\Users\\blanc\\code\\Supply_chain\\src']


In [2]:
import pandas as pd

df = pd.read_csv("../data/raw/DataCoSupplyChainDataset.csv", encoding="latin1")
df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  object 
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  object 
 9   Customer City                  180519 non-null  object 
 10  Customer Country               180519 non-null  object 
 11  Customer Email                 180519 non-null  object 
 12  Customer Fname                

In [4]:
# Type of data
from functions_preprocess import type_data
df_type = df.copy()
df_type = type_data(df_type)

In [5]:
df_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Type                           180519 non-null  category      
 1   Days for shipping (real)       180519 non-null  float64       
 2   Days for shipment (scheduled)  180519 non-null  float64       
 3   Benefit per order              180519 non-null  float64       
 4   Sales per customer             180519 non-null  float64       
 5   Delivery Status                180519 non-null  category      
 6   Late_delivery_risk             180519 non-null  float64       
 7   Category Id                    180519 non-null  float64       
 8   Category Name                  180519 non-null  category      
 9   Customer City                  180519 non-null  object        
 10  Customer Country               180519 non-null  object        
 11  

In [6]:
df_type.isnull().sum().sort_values(ascending=False)

Product Description              180519
Days for shipping (real)              0
Days for shipment (scheduled)         0
Benefit per order                     0
Type                                  0
Delivery Status                       0
Late_delivery_risk                    0
Category Id                           0
Category Name                         0
Customer City                         0
Customer Country                      0
Customer Email                        0
Customer Fname                        0
Customer Id                           0
Customer Lname                        0
Customer Password                     0
Customer Segment                      0
Customer State                        0
Customer Street                       0
Customer Zipcode                      0
Sales per customer                    0
Department Id                         0
Department Name                       0
Longitude                             0
Latitude                              0


In [7]:
# duplicates
df.duplicated().sum()

np.int64(0)

In [8]:
# Drop features and rename
cols_to_drop = [
    "Customer Email", "Customer Password", #"Customer Street",
    "Customer Fname", "Customer Lname", "Customer Zipcode",
    "Product Description", "Product Status",
    "Order Item Id", "Order Item Cardprod Id","Order Zipcode",
    "Product Card Id", "Order Customer Id", "Product Description"
]

df_type = df_type.drop(columns=cols_to_drop)


df_type = df_type.rename(columns={
    "Days for shipping (real)": "real_shipping_days",
    "Days for shipment (scheduled)": "scheduled_shipping_days",
    "Late_delivery_risk": "late_risk",
    "Order Profit Per Order": "order_profit",
    "Order Item Profit Ratio": "profit_ratio",
    "Sales per customer": "sales_per_customer"
})

In [9]:
# New features : FEATURE ENGINEERING
import numpy as np
df_type["delivery_delay"] = (
    df_type["real_shipping_days"] - df_type["scheduled_shipping_days"]
)

df_type["is_late"] = (df_type["delivery_delay"] > 0).astype(int)

df_type['order_processing_days'] = (
    df_type['shipping date (DateOrders)'] - df_type['order date (DateOrders)']
).dt.days

df_type["profit_margin"] = df_type["order_profit"] / df_type["Sales"]
df_type["profit_margin"] = df_type["profit_margin"].replace([np.inf, -np.inf], np.nan)

df_type["shipping_mode"] = df_type["Shipping Mode"].str.upper()



In [10]:
# Sauvegarde
df_type.to_csv("../data/processed/dataco_clean.csv", index=False)
print(f"Processed dataset saved")

Processed dataset saved


In [11]:
import pandas as pd
df = pd.read_csv("../data/processed/dataco_clean.csv")
df.head()

,Type,real_shipping_days,scheduled_shipping_days,Benefit per order,sales_per_customer,Delivery Status,late_risk,Category Id,Category Name,Customer City,...,Product Image,Product Name,Product Price,shipping date (DateOrders),Shipping Mode,delivery_delay,is_late,order_processing_days,profit_margin,shipping_mode
0,DEBIT,3.0,4.0,91.250000,314.640015,Advance shipping,0.0,73.0,Sporting Goods,Caguas,...,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,2018-02-03 22:56:00,Standard Class,-1.0,0,3,0.278413,STANDARD CLASS
1,TRANSFER,5.0,4.0,-249.089996,311.359985,Late delivery,1.0,73.0,Sporting Goods,Caguas,...,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,2018-01-18 12:27:00,Standard Class,1.0,1,5,-0.760000,STANDARD CLASS
2,CASH,4.0,4.0,-247.779999,309.720001,Shipping on time,0.0,73.0,Sporting Goods,San Jose,...,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,2018-01-17 12:06:00,Standard Class,0.0,0,4,-0.756003,STANDARD CLASS
3,DEBIT,3.0,4.0,22.860001,304.809998,Advance shipping,0.0,73.0,Sporting Goods,Los Angeles,...,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,2018-01-16 11:45:00,Standard Class,-1.0,0,3,0.069748,STANDARD CLASS
4,PAYMENT,2.0,4.0,134.210007,298.250000,Advance shipping,0.0,73.0,Sporting Goods,Caguas,...,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,2018-01-15 11:24:00,Standard Class,-2.0,0,2,0.409489,STANDARD CLASS


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 50 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Type                        180519 non-null  object 
 1   real_shipping_days          180519 non-null  float64
 2   scheduled_shipping_days     180519 non-null  float64
 3   Benefit per order           180519 non-null  float64
 4   sales_per_customer          180519 non-null  float64
 5   Delivery Status             180519 non-null  object 
 6   late_risk                   180519 non-null  float64
 7   Category Id                 180519 non-null  float64
 8   Category Name               180519 non-null  object 
 9   Customer City               180519 non-null  object 
 10  Customer Country            180519 non-null  object 
 11  Customer Id                 180519 non-null  float64
 12  Customer Segment            180519 non-null  object 
 13  Customer State

In [12]:
from auth_db import init_db, create_user

# ⚡ INIT DB
init_db()

# 👑 Créer un patron (tu pourras te connecter avec)
create_user(
    first="Jean",
    last="Dupont",
    username="patron1",
    email="patron1@dataco.com",
    password="Patron2025",
    role="patron"
)

# 🧑‍💼 Créer un manager
create_user(
    first="Alice",
    last="Martin",
    username="manager1",
    email="manager1@dataco.com",
    password="Manager2025",
    role="manager"
)

print("✅ Patron et manager créés avec succès")

✅ Patron et manager créés avec succès
